# Algoritmy Machine Learning
- využijeme připravené algoritmy knihovny `scikit-learn`
- cílem je
  - ukázat si použití jednotlivých algoritmů
  - ověřit si vliv přípravy vstupních a tréninkových dat


## Import knihoven a konstanty

In [1]:
import os
import pandas
import matplotlib.pyplot as plt

In [3]:
__file__ = __vsc_ipynb_file__  
SCRIPT_FOLDER = os.path.dirname(__file__)
PROJECT_FOLDER = os.path.dirname(SCRIPT_FOLDER)
kskore_FOLDER = os.path.join(PROJECT_FOLDER,'data')
WORK_FOLDER = os.path.join(kskore_FOLDER,'working')

PRQT = os.path.join(WORK_FOLDER,'roboti.prqt')

COL_ROBOT = 'robot'
COL_TIME = 'cas_robota'
COL_DAY = 'day'
COL_SENZOR = 'senzor'
COL_ZPRAVA = 'zprava'

COLS_KOSTKY = ['kostka1','kostka2','kostka3','kostka4','kostka5','kostka6']

def kostky_score(row):
    '''
    vypočítá skóre hodu 6 kostkami a vrátí výsledek
    Výpočet má následující pravidla:
    1. hodnotí se jen postupky začínající jedničkou /ostatní čísla nehrají roli/
    2. body za postupky jsou následující:
    | Postupka     | Body |
    |--------------|------|
    | 1-2          |  20  |
    | 1-2-3        |  30  |
    | 1-2-3-4      |  50  |
    | 1-2-3-4-5    |  90  |
    | 1-2-3-4-5-6  |  200  |
    
    Parameters
    ----------
    throw: neuspořádaný seznam hodnot kostek 
    
    Returns
    -------
    numeric
    
    '''
    throw = (row['kostka1'],row['kostka2'],row['kostka3'],row['kostka4'],row['kostka5'],row['kostka6'])
    score = 0
    ########## prostor pro splnění úkolu #########

    if len(throw) == 6 and 1 in throw and 2 in throw:
        score = 20
        if 3 in throw:
            score = 30
            if 4 in throw:
                score = 50
                if 5 in throw:
                    score = 90
                    if 6 in throw:
                        score = 200

    ##############################################
    return score



## Sada algoritmu `scikit-learn`

In [4]:
from sklearn.neighbors import KNeighborsClassifier,KNeighborsRegressor
from sklearn.ensemble import RandomForestClassifier,ExtraTreesClassifier,ExtraTreesRegressor,RandomForestRegressor
from sklearn.tree import DecisionTreeClassifier,ExtraTreeClassifier,DecisionTreeRegressor
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder,LabelEncoder
from sklearn.model_selection import train_test_split,TimeSeriesSplit
from sklearn.linear_model import LinearRegression,Ridge,Lasso
from sklearn.isotonic import IsotonicRegression
from sklearn.feature_extraction.text import CountVectorizer

from skforecast.recursive import ForecasterRecursive
from skforecast.direct import ForecasterDirect

CLASSIFIERS_MCMO = {    # Support multiclass-multioutput
	'DecisionTreeClassifier': DecisionTreeClassifier(),
	'ExtraTreeClassifier': ExtraTreeClassifier(),
	'ExtraTreesClassifier': ExtraTreesClassifier(),
	'KNeighborsClassifier': KNeighborsClassifier(3),
	#'RadiusNeighborsClassifier': RadiusNeighborsClassifier(),
	'RandomForestClassifier': RandomForestClassifier(max_depth=5, n_estimators=10, max_features=6, random_state=42)
}

REGRESSORS = {
    'LinearRegression': LinearRegression(),
    #'IsotonicRegression': IsotonicRegression(out_of_bounds="clip"),
    'Ridge': Ridge(),
    'KNeighborsRegressor': KNeighborsRegressor(n_neighbors=4,weights = 'uniform'),
    'DecisionTreeRegressor': DecisionTreeRegressor(max_depth=10),
    'ExtraTreesRegressor': ExtraTreesRegressor(max_depth=10),
	'RandomForestRegressor': RandomForestRegressor(max_depth=5, n_estimators=10, max_features=6, random_state=42)
}


c:\Users\Dorota\Desktop\datová analytika\working\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Získání zdrojových dat
### Přečtení dat robotů

In [5]:
all_robots = pandas.read_parquet(PRQT)
all_robots

cas_robota  \
robot   minute                                           
martinr 2025-10-23 21:08:00 2025-10-23 21:08:24.597380   
        2025-10-23 21:43:00 2025-10-23 21:43:30.880500   
        2025-10-23 21:44:00 2025-10-23 21:44:31.723900   
        2025-10-23 21:45:00 2025-10-23 21:45:37.731600   
        2025-10-23 21:50:00 2025-10-23 21:50:10.807800   
...                                                ...   
kucerak 2025-11-03 18:44:00 2025-11-03 18:44:50.105300   
        2025-11-03 18:46:00 2025-11-03 18:46:37.473200   
        2025-11-03 18:50:00 2025-11-03 18:50:59.547900   
        2025-11-03 18:52:00 2025-11-03 18:52:16.821700   
        2025-11-03 18:56:00 2025-11-03 18:56:59.056800   

                                                                        zprava  \
robot   minute                                                                   
martinr 2025-10-23 21:08:00                          Rip: start robotovy cesty   
        2025-10-23 21:43:00  kapsa slinta obrázek výstřel bublanina papírni...   
        2025-10-23 21:44:00  rovnátka odvaha rámus rýma modelka okno výtok ...   
        2025-10-23 21:45:00  postel odpad katapult guláš domov počasí obráz...   
        2025-10-23 21:50:00  liška šepot večírek morálka lusknutí čepice cí...   
...                                                                        ...   
kucerak 2025-11-03 18:44:00  strana prales rekreace marketing šepot vidle k...   
        2025-11-03 18:46:00  lehkost zrození intuice kruton výzkum díra ško...   
        2025-11-03 18:50:00  světice vlčice hrudka úředník pošťák komín pop...   
        2025-11-03 18:52:00  přístroj záložka díra rozhodčí pomeranč cestop...   
        2025-11-03 18:56:00  úskalí pivnice chamtivost burčák výzkum tykev ...   

                                  lati      longi  kostka1  kostka2  kostka3  \
robot   minute                                                                 
martinr 2025-10-23 21:08:00  50.306888  14.289606        3        3        2   
        2025-10-23 21:43:00  50.306620  14.291078        5        5        1   
        2025-10-23 21:44:00  50.306729  14.291234        6        6        1   
        2025-10-23 21:45:00  50.306935  14.291732        3        4        2   
        2025-10-23 21:50:00  50.307039  14.291918        2        5        5   
...                                ...        ...      ...      ...      ...   
kucerak 2025-11-03 18:44:00  50.398040  14.081538        4        5        4   
        2025-11-03 18:46:00  50.398147  14.082520        5        5        1   
        2025-11-03 18:50:00  50.397894  14.084559        4        3        4   
        2025-11-03 18:52:00  50.397604  14.087162        1        3        5   
        2025-11-03 18:56:00  50.397556  14.088194        6        4        4   

                             kostka4  kostka5  kostka6      cena  pocitadlo  \
robot   minute                                                                
martinr 2025-10-23 21:08:00        3        2        2  2054.169        973   
        2025-10-23 21:43:00        1        4        2  2073.273       1859   
        2025-10-23 21:44:00        3        3        6  2036.783       2661   
        2025-10-23 21:45:00        2        2        5  2009.490       3492   
        2025-10-23 21:50:00        1        2        6  2014.313       4305   
...                              ...      ...      ...       ...        ...   
kucerak 2025-11-03 18:44:00        5        5        1    43.664    6154771   
        2025-11-03 18:46:00        5        5        1    44.074    6155792   
        2025-11-03 18:50:00        1        2        1    43.206    6156853   
        2025-11-03 18:52:00        4        4        3    42.700    6157924   
        2025-11-03 18:56:00        4        3        1    42.551    6158932   

                                  senzor         day  
robot   minute                                        
martinr 2025-10-23 21:08:00  1019.392556  2

### Extrakce dat vlastního robota

In [ ]:
#robot = all_robots.loc['dorotan']
robot = all_robots.loc['martinr']
robot

,cas_robota,zprava,lati,longi,kostka1,kostka2,kostka3,kostka4,kostka5,kostka6,cena,pocitadlo,senzor,day
minute,,,,,,,,,,,,,,
2025-10-23 21:08:00,2025-10-23 21:08:24.597380,Rip: start robotovy cesty,50.306888,14.289606,3,3,2,3,2,2,2054.169,973,1019.392556,2025-10-23
2025-10-23 21:43:00,2025-10-23 21:43:30.880500,kapsa slinta obrázek výstřel bublanina papírni...,50.306620,14.291078,5,5,1,1,4,2,2073.273,1859,885.593550,2025-10-23
2025-10-23 21:44:00,2025-10-23 21:44:31.723900,rovnátka odvaha rámus rýma modelka okno výtok ...,50.306729,14.291234,6,6,1,3,3,6,2036.783,2661,802.114208,2025-10-23
2025-10-23 21:45:00,2025-10-23 21:45:37.731600,postel odpad katapult guláš domov počasí obráz...,50.306935,14.291732,3,4,2,2,2,5,2009.490,3492,831.269343,2025-10-23
2025-10-23 21:50:00,2025-10-23 21:50:10.807800,liška šepot večírek morálka lusknutí čepice cí...,50.307039,14.291918,2,5,5,1,2,6,2014.313,4305,813.123212,2025-10-23
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-11-03 18:43:00,2025-11-03 18:43:12.324500,mrož bodec letiště nozdry saze kopyto plakát b...,50.574884,13.817185,3,2,1,3,1,4,310.588,6483945,1011.457373,2025-11-03
2025-11-03 18:46:00,2025-11-03 18:46:24.174400,zebra mosaz bible mandle okno kalkulačka zmizí...,50.575218,13.817228,2,2,5,2,6,6,308.787,6484997,1051.775857,2025-11-03
2025-11-03 18:48:00,2025-11-03 18:48:18.687100,peníze díra blůza slivovice liška satelit barv...,50.576230,13.816553,3,5,2,1,4,4,305.977,6486048,1051.304589,2025-11-03


## Využití klasifikátorů
Algoritmy "s učitelem", které slouží k třídění a klasifikaci dat ...

Každý algoritmus musíme nejdříve "naučit":
1. potředujeme datovou sadu se známými výsledky
2. rozdělíme data na tréninkovou a ověřovací část
3. naučíme algoritmus na tréninkové části
4. otestujeme ho na ověřovací části a zjistíme si úspěšnost učení
5. pokud je přesnost dostatečná, můžeme algoritmus použít na data s neznámým výsledkem

### Funkce, kterou budeme používat k trénování a testování algoritmů

In [10]:
def train_classifiers(data,results):
    output = {}
    X_train, X_test, Y_train, Y_test = train_test_split(data,results,test_size=len(results)//3)
    for key,engine in CLASSIFIERS_MCMO.items():
        start = pandas.Timestamp.now()
        engine.fit(X_train,Y_train)
        step1 = pandas.Timestamp.now()
        print(f"engine {key} naucen za {(step1-start).total_seconds()} sec")
        result = engine.score(X_test, Y_test)
        step2 = pandas.Timestamp.now()
        print(f"engine {key} ma skore {result} za {(step2-step1).total_seconds()} sec")
        if result > 0.99:
            output[key] = engine
        else:
            print(f"engine {key} ma moc nizko skore ... ignorujeme ho")
    return output

### Hrajeme kostky
naučíme "systém" počítat skóre postupek na datech našeho robota  
=> spočítáme skore postupek "naší" známou funkcí

In [11]:
kskore = robot.apply(kostky_score, axis=1)
print('rozložení hodnot skore:\n',kskore.value_counts())

rozložení hodnot skore:
 0      7049
20     2097
30     1426
50      884
90      482
200     186
Name: count, dtype: int64


=> otestujeme algoritmy

In [12]:
train_classifiers(robot[COLS_KOSTKY],kskore)

engine DecisionTreeClassifier naucen za 0.264186 sec
engine DecisionTreeClassifier ma skore 0.7824795842613215 za 0.068932 sec
engine DecisionTreeClassifier ma moc nizko skore ... ignorujeme ho
engine ExtraTreeClassifier naucen za 0.029295 sec
engine ExtraTreeClassifier ma skore 0.734966592427617 za 0.010434 sec
engine ExtraTreeClassifier ma moc nizko skore ... ignorujeme ho
engine ExtraTreesClassifier naucen za 5.150521 sec
engine ExtraTreesClassifier ma skore 0.8809700569166048 za 0.522396 sec
engine ExtraTreesClassifier ma moc nizko skore ... ignorujeme ho
engine KNeighborsClassifier naucen za 0.17672 sec
engine KNeighborsClassifier ma skore 0.7213560999752536 za 0.558453 sec
engine KNeighborsClassifier ma moc nizko skore ... ignorujeme ho
engine RandomForestClassifier naucen za 0.530072 sec
engine RandomForestClassifier ma skore 0.6872061370947785 za 0.023489 sec
engine RandomForestClassifier ma moc nizko skore ... ignorujeme ho


{}

Dosažená úspěšnost není uspokojivá pokusíme se tedy upravit vstupní data do podoby vhodnější pro klasifikaci.   

### Upravíme zdrojová data použitím `Vectorizer`u
a opět je rozdělíme na tréninkovou a testovací sadu

[`CountVectorizer`](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html) vytvoří mapu s počtem výskytů 'tokenu' v řetězci

In [18]:
kdata = robot['kostka1'].astype(int).astype('str')\
    +robot['kostka2'].astype(int).astype('str')\
    +robot['kostka3'].astype(int).astype('str')\
    +robot['kostka4'].astype(int).astype('str')\
    +robot['kostka5'].astype(int).astype('str')\
    +robot['kostka6'].astype(int).astype('str')
    
kdata

minute
2025-10-23 21:08:00    332322
2025-10-23 21:43:00    551142
2025-10-23 21:44:00    661336
2025-10-23 21:45:00    342225
2025-10-23 21:50:00    255126
                        ...  
2025-11-03 18:43:00    321314
2025-11-03 18:46:00    225266
2025-11-03 18:48:00    352144
2025-11-03 18:51:00    411116
2025-11-03 18:56:00    153124
Length: 12124, dtype: object

In [19]:
vectorizer = CountVectorizer(token_pattern=r"\d")
x = vectorizer.fit_transform(kdata)
vectorizer.get_feature_names_out()

array(['1', '2', '3', '4', '5', '6'], dtype=object)

In [20]:
classifiers = train_classifiers(x,kskore)
classifiers.keys()

engine DecisionTreeClassifier naucen za 0.012697 sec
engine DecisionTreeClassifier ma skore 1.0 za 0.018919 sec
engine ExtraTreeClassifier naucen za 0.028585 sec
engine ExtraTreeClassifier ma skore 1.0 za 0.046842 sec
engine ExtraTreesClassifier naucen za 3.305276 sec
engine ExtraTreesClassifier ma skore 1.0 za 0.113369 sec
engine KNeighborsClassifier naucen za 0.001971 sec
engine KNeighborsClassifier ma skore 1.0 za 4.248836 sec
engine RandomForestClassifier naucen za 0.12721 sec
engine RandomForestClassifier ma skore 0.9853996535511013 za 0.011796 sec
engine RandomForestClassifier ma moc nizko skore ... ignorujeme ho


dict_keys(['DecisionTreeClassifier', 'ExtraTreeClassifier', 'ExtraTreesClassifier', 'KNeighborsClassifier'])

### Použítí naučených algoritmů

In [21]:
alldata = all_robots['kostka1'].astype(int).astype('str')\
    +all_robots['kostka2'].astype(int).astype('str')\
    +all_robots['kostka3'].astype(int).astype('str')\
    +all_robots['kostka4'].astype(int).astype('str')\
    +all_robots['kostka5'].astype(int).astype('str')\
    +all_robots['kostka6'].astype(int).astype('str')

allx = vectorizer.transform(alldata)

In [22]:
for key,engine in classifiers.items():
    start = pandas.Timestamp.now()
    engine.predict(allx)
    print(f"engine {key} vse ohodnotil za {(pandas.Timestamp.now()-start).total_seconds()} sec")

engine DecisionTreeClassifier vse ohodnotil za 0.03667 sec
engine ExtraTreeClassifier vse ohodnotil za 0.042345 sec
engine ExtraTreesClassifier vse ohodnotil za 3.391126 sec
engine KNeighborsClassifier vse ohodnotil za 158.013035 sec


## Práce s textem

### 1. Příprava výsledků

In [13]:
jidlo = robot[COL_ZPRAVA].str.contains("perník|chlebíček|šunka|koláček|zelňačka|bublanina") 
jidlo.value_counts()

zprava
False    10063
True      2061
Name: count, dtype: int64

### 2. Příprava dat
[`CountVectorizer`](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html) vytvoří mapu s počtem výskytů 'tokenu' v řetězci

In [14]:
vectorizer = CountVectorizer()
x = vectorizer.fit_transform(robot[COL_ZPRAVA])
print(f"Pocet nalezenych slov je:", len(vectorizer.get_feature_names_out()))
vectorizer.get_feature_names_out()

Pocet nalezenych slov je: 506


array(['abstinent', 'anoda', 'anomálie', 'anonym', 'aplikace', 'armáda',
       'artefakt', 'atest', 'atletika', 'auto', 'balada', 'balonek',
       'bar', 'barvička', 'bavlna', 'beznaděj', 'bible', 'biograf',
       'bitka', 'bižuterie', 'blbost', 'bludička', 'blůza', 'bodec',
       'bodlina', 'bokovka', 'bonbon', 'bor', 'bordel', 'bouda',
       'brigádník', 'brož', 'brzda', 'bublanina', 'bufet', 'buk',
       'burčák', 'býk', 'běžec', 'břečtaň', 'bříza', 'bříško',
       'certifikát', 'cestopis', 'cesty', 'chamtivost', 'chemie',
       'chlebíček', 'chlápek', 'chrom', 'cibule', 'citrus', 'císař',
       'darebák', 'databanka', 'dekolt', 'delfín', 'demonstrace', 'deník',
       'devítka', 'deštník', 'dimenze', 'disketa', 'dlaň', 'domov',
       'dotknout', 'dovolená', 'dračice', 'drb', 'drobek', 'drzost',
       'duha', 'duplikát', 'díra', 'dítě', 'děvče', 'ekonomika', 'epocha',
       'epopej', 'estetika', 'eufórie', 'evropa', 'facebook', 'facka',
       'filtrace', 'flek', 'fyzičk

### 3. Trénování algoritmů

In [15]:
classifiers = train_classifiers(x,jidlo)

engine DecisionTreeClassifier naucen za 0.090567 sec
engine DecisionTreeClassifier ma skore 1.0 za 0.017714 sec
engine ExtraTreeClassifier naucen za 0.229262 sec
engine ExtraTreeClassifier ma skore 0.9396189062113338 za 0.010233 sec
engine ExtraTreeClassifier ma moc nizko skore ... ignorujeme ho
engine ExtraTreesClassifier naucen za 9.511077 sec
engine ExtraTreesClassifier ma skore 1.0 za 0.185754 sec
engine KNeighborsClassifier naucen za 0.003001 sec
engine KNeighborsClassifier ma skore 0.833951992081168 za 3.723283 sec
engine KNeighborsClassifier ma moc nizko skore ... ignorujeme ho
engine RandomForestClassifier naucen za 0.163486 sec
engine RandomForestClassifier ma skore 0.8247958426132146 za 0.025979 sec
engine RandomForestClassifier ma moc nizko skore ... ignorujeme ho


### Použití na všech robotech

### 1. Příprava výsledků

### 2. Příprava dat
[`CountVectorizer`](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html) vytvoří mapu s počtem výskytů 'tokenu' v řetězci

In [16]:
allx = vectorizer.transform(all_robots[COL_ZPRAVA])

### 3. Použití algoritmů

In [17]:
for key,engine in classifiers.items():
    start = pandas.Timestamp.now()
    engine.predict(allx)
    print(f"engine {key} vse ohodnotil za {(pandas.Timestamp.now()-start).total_seconds()} sec")
    

engine DecisionTreeClassifier vse ohodnotil za 0.053434 sec
engine ExtraTreesClassifier vse ohodnotil za 8.956113 sec


## Regresní algoritmy
Algoritmy, které se snaží modelovat průběh/závislosti hodnot

### Příprava dat a učení algoritmů

### Použití na celou datovou sadu

### Rozdělení na samostatné grafy
#### 1. připravíme pracovní plochu

#### 2. vygenerujeme grafy

## Predikce budoucnosti
použijeme `cenu robota`

### 1. Rozdělíme data na tréninková a testovací

### 2. Připravíme si předpovídátko

### 3. Předpovíme budoucí hodnoty